TO-DOs:

INCIDENT FILE
- Parse-out the time from the date/time columns
- Calculate the arrival times

STATION FILE
- TBD?


ASSIGNMENTS:
- Google map the CLV station on a map?  (Margot)


In [73]:
# DEPENDENCIES
import pandas as pd

# Declare variables for each CSV file
incident_file_df1114 = "data/Fire_Department_Incident_Count - 2011-2014.csv"
incident_file_df1516 = "data/Fire_Department_Incident_Count - 2015-2016.csv"
incident_file_df1718 = "data/Fire_Department_Incident_Count - 2017-2018.csv"
fire_station_data =  "data/Clark_County_Fire_Stations.csv"

PROCESS INCIDENT DATA HERE

In [ ]:
# Step 1: bring-in 2011-2014 data into a df

incident_file_df1114 = pd.read_csv(incident_file1)
print(f"# of rows in 2011-2014 DF: {len(incident_file_df1114)}")

In [17]:
# Step 2: bring-in 2015 - 2015 data into a df

incident_file_df1516 = pd.read_csv(incident_file2)
print(f"# of rows in 2014-2015 DF: {len(incident_file_df1516)}")

# of rows in 2014-2015 DF: 207549


In [18]:
# Step 3: bring-in 2016-2018 data into a df

incident_file_df1718 = pd.read_csv(incident_file3)
print(f"# of rows in 2017-2018 file: {len(incident_file_df1718)}")

# of rows in 2017-2018 file: 204040


In [33]:
# Concatonate previous DFs into a single dataframe

incident_file_df = pd.concat([incident_file_df1114, incident_file_df1516, incident_file_df1718])
print(f"# of rows in complete file: {len(incident_file_df)}")

# of rows in complete file: 729132


In [85]:
incident_file_df.count()

# Possibly delete rows with a missing Assign_Time??

Response_Date          729132
Event_Type             729132
Division_Name          729132
Station                729132
Radio_Name             729132
First_Unit_Assigned    729132
First_Unit_Arrived     708375
Assign_Time            699105
Arrive_Time            729132
Ward                   724431
Location               729132
Location_1             729132
Jurisdiction           729132
Fiscal_Year            729132
Per_Capita             729132
ObjectId               729132
dtype: int64

In [30]:
# Drop all rows with missing information
#incident_file_df = incident_file_df.dropna(how='any')

# *** I DON'T THINK WE WANT TO DO THIS ***

In [39]:
incident_file_lean_df = incident_file_df[["Station","Response_Date","Event_Type","First_Unit_Assigned","First_Unit_Arrived","Location","Location_1"]]
incident_file_lean_df.head()

,Station,Response_Date,Event_Type,First_Unit_Assigned,First_Unit_Arrived,Location,Location_1
0,Station 4,2011/07/01 00:00:00+00,Medical,2011/07/01 00:01:00+00,7/1/2011 0:06,S Maryland Pky & E Carson Ave,"(36.16462708, -115.1334076)"
1,Station 44,2011/07/01 00:05:00+00,Medical,2011/07/01 00:06:00+00,7/1/2011 0:12,100 block of Lotus Blossom Ct,"(36.17433167, -115.2525711)"
2,Station 3,2011/07/01 00:22:00+00,Fire,2011/07/01 00:24:00+00,7/1/2011 0:27,N RANCHO DR & W WASHINGTON AVE,"(36.18107986, -115.1791687)"
3,Station 10,2011/07/01 00:22:00+00,Medical,2011/07/01 00:23:00+00,7/1/2011 0:29,2200 block of S Las Vegas Blvd,"(36.1462059, -115.1557083)"
4,Other,2011/07/01 00:23:00+00,Public Need,2011/07/01 00:24:00+00,7/1/2011 0:31,7300 block of Breezy Night Ct,"(36.2447052, -115.2542267)"


In [67]:
# Create new df for rows that DO NOT contain 'none' in the First_Unit_Arrived column

incident_file_clean_df = incident_file_lean_df[incident_file_lean_df["First_Unit_Arrived"] != "None"]
incident_file_clean_df.count()

Station                682788
Response_Date          682788
Event_Type             682788
First_Unit_Assigned    682788
First_Unit_Arrived     662031
Location               682788
Location_1             682788
dtype: int64

In [68]:
incident_file_clean_df.head()

,Station,Response_Date,Event_Type,First_Unit_Assigned,First_Unit_Arrived,Location,Location_1
0,Station 4,2011/07/01 00:00:00+00,Medical,2011/07/01 00:01:00+00,7/1/2011 0:06,S Maryland Pky & E Carson Ave,"(36.16462708, -115.1334076)"
1,Station 44,2011/07/01 00:05:00+00,Medical,2011/07/01 00:06:00+00,7/1/2011 0:12,100 block of Lotus Blossom Ct,"(36.17433167, -115.2525711)"
2,Station 3,2011/07/01 00:22:00+00,Fire,2011/07/01 00:24:00+00,7/1/2011 0:27,N RANCHO DR & W WASHINGTON AVE,"(36.18107986, -115.1791687)"
3,Station 10,2011/07/01 00:22:00+00,Medical,2011/07/01 00:23:00+00,7/1/2011 0:29,2200 block of S Las Vegas Blvd,"(36.1462059, -115.1557083)"
4,Other,2011/07/01 00:23:00+00,Public Need,2011/07/01 00:24:00+00,7/1/2011 0:31,7300 block of Breezy Night Ct,"(36.2447052, -115.2542267)"


In [69]:
incident_file_clean_df["Assigned Time"] = "TBD"

<ipython-input-69-d6c3b4e438cf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incident_file_clean_df["Assigned Time"] = "TBD"


In [70]:
incident_file_clean_df.head()

,Station,Response_Date,Event_Type,First_Unit_Assigned,First_Unit_Arrived,Location,Location_1,Assigned Time
0,Station 4,2011/07/01 00:00:00+00,Medical,2011/07/01 00:01:00+00,7/1/2011 0:06,S Maryland Pky & E Carson Ave,"(36.16462708, -115.1334076)",TBD
1,Station 44,2011/07/01 00:05:00+00,Medical,2011/07/01 00:06:00+00,7/1/2011 0:12,100 block of Lotus Blossom Ct,"(36.17433167, -115.2525711)",TBD
2,Station 3,2011/07/01 00:22:00+00,Fire,2011/07/01 00:24:00+00,7/1/2011 0:27,N RANCHO DR & W WASHINGTON AVE,"(36.18107986, -115.1791687)",TBD
3,Station 10,2011/07/01 00:22:00+00,Medical,2011/07/01 00:23:00+00,7/1/2011 0:29,2200 block of S Las Vegas Blvd,"(36.1462059, -115.1557083)",TBD
4,Other,2011/07/01 00:23:00+00,Public Need,2011/07/01 00:24:00+00,7/1/2011 0:31,7300 block of Breezy Night Ct,"(36.2447052, -115.2542267)",TBD


PROCESS FIRE STATION DATA

In [84]:
# Bring-in fire station data
station_df = pd.read_csv(fire_station_data)
print(f"# of rows in the Station data file: {len(station_df)}")
#station_df.head()

# of rows in the Station data file: 89


In [83]:
distinct_stations = incident_file_clean_df["Station"].unique()
clv_stations_df = station_df[station_df["CITY_CODE"] == "CLV"]
#clv_stations_df
